In [ ]:
!pip install requests python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.6 MB/s eta 0:00:00


In [32]:
import requests
from docx import Document
import os
subscription_key = "subscription_key"
endpoint = 'https://api.cognitive.microsofttranslator.com'
location = "eastus2"
language_destination = 'pt-br'

def translator_text(text, target_language):
    path = '/translate'
    constructed_url = endpoint + path
    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key,
        'Ocp-Apim-Subscription-Region': location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(os.urandom(16))
    }

    body = [{
        'text': text
    }]
    params = {
        'api-version': '3.0',
        'from': 'en',
        'to': target_language

    }
    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()

    if isinstance(response, list) and len(response) > 0 and "translations" in response[0] and len(response[0]["translations"]) > 0 and "text" in response[0]["translations"][0]:
        return response[0]["translations"][0]["text"]
    else:
        print(f"Error: Unexpected response format: {response}")
        return None


In [33]:
translator_text("I know you're somewhere out there, somewhere far away", language_destination)

'Eu sei que você está em algum lugar lá fora, em algum lugar distante'

In [39]:
def translate_document(path):
    document = Document(path)
    full_text = []
    for paragraph in document.paragraphs:
        translateed_text = translator_text(paragraph.text, language_destination)
        full_text.append(translateed_text)

    translate_doc = Document()
    for line in full_text:
        translate_doc.add_paragraph(line)
    path_translated = path.replace(".docx", f"_{language_destination}.docx")
    translate_doc.save(path_translated)
    return path_translated

In [38]:
input_file = "/content/talking to the moon.docx"
translate_document(input_file)

'/content/talking to the moon_pt-br.docx'

In [40]:
!pip  install requests beautifulsoup4 openai langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstalling langchain-core-0.3.19:
      Successfully uninstalled langchain-core-0.3.19


In [42]:
import requests
from bs4 import BeautifulSoup

def extract_text_from_url(url):
    response = requests.get(url)

    if response.status_code == 200:
      soup = BeautifulSoup(response.text, 'html.parser')
      for script_or_style in soup(["script", "style"]):
        script_or_style.decompose()
      text = soup.get_text(separator=' ')
      #Limpar texto
      lines = (line.strip() for line in text.splitlines())
      chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
      clean_text = '\n'.join(chunk for chunk in chunks if chunk)
      return clean_text
    else:
      print(f"Failed to fetch the URL. Status code: {response.status_code}")
      return None

    text = soup.get_text()
    return text

extract_text_from_url("https://dev.to/getvm/distributed-systems-mastering-the-art-of-complex-computing-networks-31pc")

'Distributed Systems: Mastering the Art of Complex Computing Networks - DEV Community\nSkip to content\nNavigation menu\nSearch\nPowered by\nSearch\nAlgolia\nSearch\nLog in\nCreate account\nDEV Community\nClose\nAdd reaction\nLike\nUnicorn\nExploding Head\nRaised Hands\nFire\nJump to Comments\nSave\nBoost\nMore...\nCopy link\nCopy link\nCopied to Clipboard\nShare to X\nShare to LinkedIn\nShare to Facebook\nShare to Mastodon\nReport Abuse\nGetVM\nPosted on\nNov 30\nDistributed Systems: Mastering the Art of Complex Computing Networks\n# getvm\n# programming\n# tutorials\nTutorials (106 Part Series)\n1\nDive into Computer Science with These Free Online Tutorials 🖥️\n2\nDive into the World of Machine Learning with GetVM\'s Free Tutorials\n...\n102 more parts...\n3\nMastering the Unix Universe: A Comprehensive Guide to Free Programming Resources\n4\nDive into the World of Programming with These Free Resources 🌐\n5\nDive Into JavaScript: A Comprehensive Learning Journey\n6\nMastering CSS: A 

In [47]:
from langchain_openai.chat_models.azure import AzureChatOpenAI

client = AzureChatOpenAI(
    azure_endpoint= "https://oai-dio-bootcamp-dev-eastus2-001-bruna.openai.azure.com/",
    api_key= "api_key",
    api_version= "2024-02-15-preview",
    deployment_name= "gpt-4o-mini",
    max_retries= 0
)

def translate_article(text, lang):
  messages = [
      ("system", "Você atua como tradutor de textos"),
      ("user", f"Traduza o {text} para o idioma {lang} e responda em markdown")
  ]

  response = client.invoke(messages)
  print(response.content)
  return response.content

translate_article("Students learn critical techniques", "portugues")

A tradução de "Students learn critical techniques" para o português é:

```markdown
Os alunos aprendem técnicas críticas.
```


'A tradução de "Students learn critical techniques" para o português é:\n\n```markdown\nOs alunos aprendem técnicas críticas.\n```'

In [50]:
url = "https://dev.to/kenakamu/azure-open-ai-in-vnet-3alo"
text = extract_text_from_url(url)
article = translate_article(text, "portugues")

print(article)

RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-15-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 86400 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}